* 단어를 벡터로 만드는 또 다른 방법으로 페이스북에서 개발한 FastText
* Word2Vec 매커니즘의 확장
* Word2Vec는 단어를 쪼개질 수 없는 단위로 생각한다면, FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주

## 1. 내부 단어(subword)의 학습
* FastText에서는 각 단어는 글자 단위 n-gram의 구성으로 취급
* n을 몇으로 결정하는지에 따라 단어들이 얼마나 분리되는지 결정된다.
    * n=3인 경우, 'apple'
    
    ```<ap, app, ppl, ple, le>, <apple>```
    * n = 3 ~ 6인 경우(default값), 'apple'
    
    ```<ap, app, ppl, ppl, le>, <app, appl, pple, ple>, <appl, pple>, ..., <apple>```
* 위와 같이 내부 단어들의 벡터값의 총 합으로 구성
```apple = <ap + app + ppl + ppl + le> + <app + appl + pple + ple> + <appl + pple> + , ..., +<apple>``

## 2. 모르는 단어(Out Of Vocabulary, OOV)에 대한 대응
FastText의 인공 신경망을 학습한 후에는 데이터 셋의 모든 단어의 각 n-gram에 대해 워드 임베딩이 된다. 이렇게 되면 데이터 셋만 충분하다면 위와 같은 내부 단어(Subword)를 통해 모르는 단어(Out of Vocabulary, OOV)에 대해 다른 단어와의 유사도를 계산할 수 있다.

## 3. 단어 집합 내 빈도 수가 적었던 단어(Rare Word)에 대한 대응
만약 단어가 희귀 단어라도, 그 단어의 n-gram이 다른 단어의 n-gram과 겹치는 경우라면, Word2Vec과 비교하여 비교적 높은 임베딩 벡터값을 얻는다. 그래서 오타가 섞인 단어에 대해서도 일정 수준의 성능을 보인다.

## 4. 실습으로 비교하는 Word2Vec Vs. FastText

In [1]:
# 훈련 데이터 다운로드 및 전처리
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

C:\Users\A\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\A\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\A\anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1c6eb2fe608>)

### 1) Word2Vec
학습 데이터에 존재하지 않는 단어의 유사도를 계산할 수 없다.

In [3]:
# 데이터 전처리


targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF-8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
    
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [5]:
from gensim.models import KeyedVectors
loaded_model = KeyedVectors.load_word2vec_format("./data/eng_w2v") # 모델 로드

In [7]:
loaded_model.wv.most_similar("electrofishing")

C:\Users\A\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


KeyError: "word 'electrofishing' not in vocabulary"

### 2) FastText

In [8]:
from gensim.models import FastText
model = FastText(result, size=100, window=5, min_count=5, workers=4, sg=1)

In [9]:
model.wv.most_similar("electrofishing")
# 유사한 단어를 계산해서 출력

[('electrolux', 0.7808310985565186),
 ('electro', 0.7780073881149292),
 ('electrolyte', 0.7718387842178345),
 ('gastric', 0.7707589864730835),
 ('electric', 0.7638455033302307),
 ('electroshock', 0.759493350982666),
 ('petroleum', 0.7573997974395752),
 ('electrochemical', 0.7540666460990906),
 ('overfishing', 0.7448238730430603),
 ('fishing', 0.7374414205551147)]

## 5. 한국어에서의 FastText
### (1) 음절 단위
n=3, '자연어처리'
```<자연, 자연어, 연어처, 어처리, 처리>```

### (2) 자모 단위
더 나아가 자모 단위(초성, 중성, 종성 단위)로 임베딩하는 시도가 있었다.
음절 단위가 아니라, 자모 단위로 가게 되면 오타나 노이즈 측면에서 더 강한 임베딩을 기대해볼 수 있다.

```분리된 결과 : ㅈ ㅏ _ ㅇ ㅕ ㄴ ㅇ ㅓ _ ㅊ ㅓ _ ㄹ ㅣ _```

n=3일때,

```< ㅈ ㅏ, ㅈ ㅏ _, ㅏ _ ㅇ, ... 중략>```